In [1]:
import pandas as pd
df=pd.read_csv('/Users/chidam/Desktop/Elo Merchant Category Recommendation/historical_transactions.csv')

In [2]:
df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [3]:
len(df)

29112361

In [ ]:
201917+

In [4]:
#check for missing values. there are a lot of values missing in category_2
df.isnull().sum()*100/len(df)

authorized_flag         0.000000
card_id                 0.000000
city_id                 0.000000
category_1              0.000000
installments            0.000000
category_3              0.611970
merchant_category_id    0.000000
merchant_id             0.475678
month_lag               0.000000
purchase_amount         0.000000
purchase_date           0.000000
category_2              9.112500
state_id                0.000000
subsector_id            0.000000
dtype: float64

In [7]:
#digging in deeper with category_2. Let's check for the unique values in the column
# Turns out category_2 is an anonymized category-data dictionary

df['category_2'].value_counts(dropna=False)

1.0    15177199
3.0     3911795
5.0     3725915
NaN     2652864
4.0     2618053
2.0     1026535
Name: category_2, dtype: int64

In [ ]:
#approx. three months worth of transaction details, so probably have to group by card_id

#before that let me check the training dataset and other datasets



In [8]:
df_train=pd.read_csv('/Users/chidam/Desktop/Elo Merchant Category Recommendation/train.csv')

In [9]:
#the target here is what I am trying to predict
df_train.head()

#would love to know how the target column was calculated? Because I am trying to predict continuous values, this is
# a regression problem

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [10]:
len(df_train)

201917

In [11]:
#check for missing values. there are no values missing

df_train.isnull().sum()*100/len(df_train)

first_active_month    0.0
card_id               0.0
feature_1             0.0
feature_2             0.0
feature_3             0.0
target                0.0
dtype: float64

In [ ]:
there is no info provided in the data sictionary for feature_1, feature_2 and feature_3

In [12]:
#ought to check whether there are multiple values for every card in the training set?

df_train['card_id'].value_counts()

C_ID_70a25df9e6    1
C_ID_9302af3146    1
C_ID_66256e3731    1
C_ID_f796ffe353    1
C_ID_0e12186a88    1
                  ..
C_ID_bac30faf42    1
C_ID_6c1b78110a    1
C_ID_28b8bab7e2    1
C_ID_3783d194d4    1
C_ID_5b14ba8add    1
Name: card_id, Length: 201917, dtype: int64

In [15]:
#there are no duplicate values

print(any(df_train['card_id'].duplicated()))   

False


In [17]:
df_train['first_active_month'] = pd.to_datetime(df_train['first_active_month'])

In [22]:
#list of unique years represented in the column
df_train['first_active_month'].dt.year.unique()

array([2017, 2016, 2015, 2013, 2014, 2012, 2011, 2018])

In [23]:
df_merchant=pd.read_csv('/Users/chidam/Desktop/Elo Merchant Category Recommendation/merchants.csv')


In [24]:
df_merchant.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [25]:
#Again the category_2 pops up with more number of missing values
df_merchant.isnull().sum()*100/len(df_merchant)

merchant_id                    0.000000
merchant_group_id              0.000000
merchant_category_id           0.000000
subsector_id                   0.000000
numerical_1                    0.000000
numerical_2                    0.000000
category_1                     0.000000
most_recent_sales_range        0.000000
most_recent_purchases_range    0.000000
avg_sales_lag3                 0.003884
avg_purchases_lag3             0.000000
active_months_lag3             0.000000
avg_sales_lag6                 0.003884
avg_purchases_lag6             0.000000
active_months_lag6             0.000000
avg_sales_lag12                0.003884
avg_purchases_lag12            0.000000
active_months_lag12            0.000000
category_4                     0.000000
city_id                        0.000000
state_id                       0.000000
category_2                     3.551581
dtype: float64

In [26]:
df_new_merchant=pd.read_csv('/Users/chidam/Desktop/Elo Merchant Category Recommendation/new_merchant_transactions.csv')

In [27]:
df_new_merchant.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,3/11/18 14:57,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,3/19/18 18:53,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,4/26/18 14:08,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671926,3/7/18 9:43,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,3/22/18 21:07,NaN,-1,29


In [28]:
df_test=pd.read_csv('/Users/chidam/Desktop/Elo Merchant Category Recommendation/test.csv')


In [29]:
df_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [32]:
len(df_test)

123623

In [35]:
df_test['feature_3'].value_counts(dropna=False)

1    69770
0    53853
Name: feature_3, dtype: int64

In [36]:
df_train['feature_3'].value_counts(dropna=False)

1    114198
0     87719
Name: feature_3, dtype: int64

In [37]:
#not sure whether the values in the column are ordinal (ordinal could be applied: to numerical values that should be treated as
#as categorical ones. So 2 > 1)

df_train['feature_1'].value_counts(dropna=False)

3    73573
2    55797
5    40625
4    19885
1    12037
Name: feature_1, dtype: int64

In [38]:
df_train['feature_2'].value_counts(dropna=False)

1    89242
2    74839
3    37836
Name: feature_2, dtype: int64

In [30]:
#first ought to check whether I have the details in historical_transactions, new_merchant_transactions, and 
#merchants for most of the card_id in the training and test sets
#First, I am checking whether the card_id in the test set also exist in the historical_transactions dataset. And, yes,
# all the card_id in the test set also present in the training set

df_test['card_id'].isin(df['card_id']).value_counts()

True    123623
Name: card_id, dtype: int64

In [33]:
#Now let's check whether the same is true for the training set. Like the test set, the training set has card_id that are 
#also present in the historical_transactions set


df_train['card_id'].isin(df['card_id']).value_counts()



True    201917
Name: card_id, dtype: int64

In [34]:
There are a lot more records in the historical_transactions dataset

len(df_train)+len(df_test)=201917+123623

325540